In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer

In [2]:
mvp = pd.read_csv("Data/mvp.csv")
current_pg = pd.read_csv("Data/pergame.csv")
current_adv = pd.read_csv("Data/advanced.csv")

In [3]:
mvp

,Season,League,Player,Voting,Age,Team,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48
0,2020,NBA,Giannis Antetokounmpo,(V),25,MIL,63,30.4,29.5,13.6,5.6,1.0,1.0,0.553,0.304,0.633,11.1,0.279
1,2019,NBA,Giannis Antetokounmpo,(V),24,MIL,72,32.8,27.7,12.5,5.9,1.3,1.5,0.578,0.256,0.729,14.4,0.292
2,2018,NBA,James Harden,(V),28,HOU,72,35.4,30.4,5.4,8.8,1.8,0.7,0.449,0.367,0.858,15.4,0.289
3,2017,NBA,Russell Westbrook,(V),28,OKC,81,34.6,31.6,10.7,10.4,1.6,0.4,0.425,0.343,0.845,13.1,0.224
4,2016,NBA,Stephen Curry,(V),27,GSW,79,34.2,30.1,5.4,6.7,2.1,0.2,0.504,0.454,0.908,17.9,0.318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,1960,NBA,Wilt Chamberlain,(V),23,PHW,72,46.4,37.6,27.0,2.3,NaN,NaN,0.461,NaN,0.582,17.0,0.245
61,1959,NBA,Bob Pettit,(V),26,STL,72,39.9,29.2,16.4,3.1,NaN,NaN,0.438,NaN,0.759,14.8,0.246
62,1958,NBA,Bill Russell,(V),23,BOS,69,38.3,16.6,22.7,2.9,NaN,NaN,0.442,NaN,0.519,11.3,0.206
63,1957,NBA,Bob Cousy,(V),28,BOS,64,36.9,20.6,4.8,7.5,NaN,NaN,0.378,NaN,0.821,8.8,0.178


In [4]:
current_pg = current_pg.drop(["Pos","GS","FG", "FGA", "ORB", "DRB", "3P", "3PA", "2P", "2PA", "2P%", "FT", "FTA", "TOV", "PF", "eFG%"], axis=1)
current_pg

,Rk,player,age,Team,g,mp_per_g,fg_pct,fg3_pct,ft_pct,trb_per_g,ast_per_g,stl_per_g,blk_per_g,pts_per_g
0,1,Stephen Curry,32,GSW,56,34.0,0.486,0.426,0.916,5.5,5.8,1.2,0.1,31.3
1,2,Bradley Beal,27,WAS,55,35.5,0.484,0.347,0.901,4.7,4.5,1.2,0.3,31.0
2,3,Luka Dončić,21,DAL,57,35.2,0.486,0.356,0.725,8.0,8.9,1.0,0.6,28.6
3,4,Damian Lillard,30,POR,58,35.8,0.435,0.378,0.926,4.2,7.5,0.9,0.3,28.3
4,5,Giannis Antetokounmpo,26,MIL,53,33.0,0.568,0.305,0.684,11.2,5.9,1.2,1.2,28.0
5,6,Zach LaVine,25,CHI,53,35.2,0.506,0.416,0.848,5.1,5.1,0.8,0.5,27.5
6,7,Zion Williamson,20,NOP,59,33.1,0.616,0.313,0.694,7.1,3.7,0.9,0.6,27.0
7,8,Kyrie Irving,28,BRK,47,35.2,0.502,0.384,0.924,4.9,6.3,1.3,0.6,26.9
8,9,Donovan Mitchell,24,UTA,53,33.4,0.438,0.386,0.845,4.4,5.2,1.0,0.3,26.4
9,10,Jayson Tatum,22,BOS,57,35.9,0.458,0.388,0.870,7.5,4.4,1.2,0.4,26.4


In [5]:
current_adv = current_adv.drop(["Rk", "Pos", "age", "Team", "g", "MP", "TS%", "3PAr", "FTr", "ORB%", "DRB%", "TOV%", "TRB%", "AST%", "STL%", "BLK%", "OBPM", "DBPM", "BPM", "VORP"], axis=1)
current_adv

,player,PER,USG%,OWS,DWS,ws,ws_per_48
0,Giannis Antetokounmpo,29.1,32.6,5.8,3.1,8.9,0.244
1,Bradley Beal,22.9,34.3,3.9,1.5,5.5,0.134
2,Devin Booker,19.3,32.8,2.4,2.0,4.4,0.106
3,Stephen Curry,26.1,34.0,5.8,2.1,7.9,0.199
4,Luka Dončić,26.0,35.8,5.0,2.3,7.4,0.176
5,De'Aaron Fox,20.8,31.0,3.6,0.4,4.0,0.093
6,Kyrie Irving,24.2,30.5,5.1,1.3,6.4,0.185
7,Nikola Jokić,31.1,29.2,10.9,3.1,14.0,0.300
8,Zach LaVine,21.6,31.2,3.6,1.6,5.2,0.133
9,Kawhi Leonard,26.8,29.0,6.0,2.1,8.2,0.243


In [6]:
all_curr_data = pd.merge(left=current_pg, right=current_adv, left_on="player", right_on="player")
all_curr_data

,Rk,player,age,Team,g,mp_per_g,fg_pct,fg3_pct,ft_pct,trb_per_g,ast_per_g,stl_per_g,blk_per_g,pts_per_g,PER,USG%,OWS,DWS,ws,ws_per_48
0,1,Stephen Curry,32,GSW,56,34.0,0.486,0.426,0.916,5.5,5.8,1.2,0.1,31.3,26.1,34.0,5.8,2.1,7.9,0.199
1,2,Bradley Beal,27,WAS,55,35.5,0.484,0.347,0.901,4.7,4.5,1.2,0.3,31.0,22.9,34.3,3.9,1.5,5.5,0.134
2,3,Luka Dončić,21,DAL,57,35.2,0.486,0.356,0.725,8.0,8.9,1.0,0.6,28.6,26.0,35.8,5.0,2.3,7.4,0.176
3,4,Damian Lillard,30,POR,58,35.8,0.435,0.378,0.926,4.2,7.5,0.9,0.3,28.3,24.5,31.7,7.5,0.6,8.1,0.187
4,5,Giannis Antetokounmpo,26,MIL,53,33.0,0.568,0.305,0.684,11.2,5.9,1.2,1.2,28.0,29.1,32.6,5.8,3.1,8.9,0.244
5,6,Zach LaVine,25,CHI,53,35.2,0.506,0.416,0.848,5.1,5.1,0.8,0.5,27.5,21.6,31.2,3.6,1.6,5.2,0.133
6,7,Zion Williamson,20,NOP,59,33.1,0.616,0.313,0.694,7.1,3.7,0.9,0.6,27.0,27.4,29.8,7.2,1.4,8.6,0.212
7,8,Kyrie Irving,28,BRK,47,35.2,0.502,0.384,0.924,4.9,6.3,1.3,0.6,26.9,24.2,30.5,5.1,1.3,6.4,0.185
8,9,Donovan Mitchell,24,UTA,53,33.4,0.438,0.386,0.845,4.4,5.2,1.0,0.3,26.4,21.3,33.6,4.0,2.2,6.2,0.167
9,10,Jayson Tatum,22,BOS,57,35.9,0.458,0.388,0.870,7.5,4.4,1.2,0.4,26.4,21.3,30.7,3.8,2.2,6.0,0.140


In [7]:
csv = os.listdir("Data")

In [8]:
csv

['1986.csv',
 '2008.csv',
 '1983.csv',
 '1962.csv',
 '2017.csv',
 '1991.csv',
 '1964.csv',
 '2014.csv',
 'mvp.csv',
 '1995.csv',
 '1994.csv',
 '1966.csv',
 '1960.csv',
 '2015.csv',
 '2018.csv',
 'advanced.csv',
 '2009.csv',
 '1977.csv',
 '1984.csv',
 '1985.csv',
 '2012.csv',
 '2013.csv',
 '1979.csv',
 '2001.csv',
 '1967.csv',
 '1996.csv',
 '1965.csv',
 '1990.csv',
 '2010.csv',
 '1987.csv',
 '1961.csv',
 '1993.csv',
 '1981.csv',
 '2016.csv',
 '2020.csv',
 '1980.csv',
 '2002.csv',
 '1998.csv',
 '2011.csv',
 '2003.csv',
 '2019.csv',
 'pergame.csv',
 '2004.csv',
 '2005.csv',
 '1999.csv',
 '2006.csv',
 '1997.csv',
 '2000.csv',
 '1963.csv',
 '1989.csv',
 '1982.csv',
 '1978.csv',
 '1956.csv',
 '.ipynb_checkpoints',
 '1958.csv',
 '1988.csv',
 '1992.csv',
 '2007.csv',
 '1959.csv',
 '1957.csv']

In [9]:
test = pd.read_csv("Data/2020.csv")

In [10]:
test

,player,age,team_id,votes_first,points_won,points_max,award_share,g,mp_per_g,pts_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,fg_pct,fg3_pct,ft_pct,ws,ws_per_48
0,Giannis Antetokounmpo,25,MIL,85.0,962.0,1010,0.952,63,30.4,29.5,13.6,5.6,1.0,1.0,0.553,0.304,0.633,11.1,0.279
1,LeBron James,35,LAL,16.0,753.0,1010,0.746,67,34.6,25.3,7.8,10.2,1.2,0.5,0.493,0.348,0.693,9.8,0.204
2,James Harden,30,HOU,0.0,367.0,1010,0.363,68,36.5,34.3,6.6,7.5,1.8,0.9,0.444,0.355,0.865,13.1,0.254
3,Luka Dončić,20,DAL,0.0,200.0,1010,0.198,61,33.6,28.8,9.4,8.8,1.0,0.2,0.463,0.316,0.758,8.8,0.207
4,Kawhi Leonard,28,LAC,0.0,168.0,1010,0.166,57,32.4,27.1,7.1,4.9,1.8,0.6,0.470,0.378,0.886,8.7,0.226
5,Anthony Davis,26,LAL,0.0,82.0,1010,0.081,62,34.4,26.1,9.3,3.2,1.5,2.3,0.503,0.330,0.846,11.1,0.250
6,Chris Paul,34,OKC,0.0,26.0,1010,0.026,70,31.5,17.6,5.0,6.7,1.6,0.2,0.489,0.365,0.907,8.9,0.193
7,Damian Lillard,29,POR,0.0,23.0,1010,0.023,66,37.5,30.0,4.3,8.0,1.1,0.3,0.463,0.401,0.888,11.6,0.225
8,Nikola Jokić,24,DEN,0.0,18.0,1010,0.018,73,32.0,19.9,9.7,7.0,1.2,0.6,0.528,0.314,0.817,9.8,0.202
9,Pascal Siakam,25,TOR,0.0,17.0,1010,0.017,60,35.2,22.9,7.3,3.5,1.0,0.9,0.453,0.359,0.792,5.4,0.123


In [11]:
def vote_predict(train_yr):
    prev_df = pd.read_csv(os.path.join("Data", str(train_yr) + ".csv"))
    return prev_df

In [12]:
twenty = vote_predict(2020)

In [13]:
nine = vote_predict(2019)

In [14]:
frames = [nine, twenty]
pd.concat(frames)

,player,age,team_id,votes_first,points_won,points_max,award_share,g,mp_per_g,pts_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,fg_pct,fg3_pct,ft_pct,ws,ws_per_48
0,Giannis Antetokounmpo,24,MIL,78.0,941.0,1010,0.932,72,32.8,27.7,12.5,5.9,1.3,1.5,0.578,0.256,0.729,14.4,0.292
1,James Harden,29,HOU,23.0,776.0,1010,0.768,78,36.8,36.1,6.6,7.5,2.0,0.7,0.442,0.368,0.879,15.2,0.254
2,Paul George,28,OKC,0.0,356.0,1010,0.352,77,36.9,28.0,8.2,4.1,2.2,0.4,0.438,0.386,0.839,11.9,0.201
3,Nikola Jokić,23,DEN,0.0,212.0,1010,0.210,80,31.3,20.1,10.8,7.3,1.4,0.7,0.511,0.307,0.821,11.8,0.226
4,Stephen Curry,30,GSW,0.0,175.0,1010,0.173,69,33.8,27.3,5.3,5.2,1.3,0.4,0.472,0.437,0.916,9.7,0.199
5,Damian Lillard,28,POR,0.0,69.0,1010,0.068,80,35.5,25.8,4.6,6.9,1.1,0.4,0.444,0.369,0.912,12.1,0.205
6,Joel Embiid,24,PHI,0.0,49.0,1010,0.049,64,33.7,27.5,13.6,3.7,0.7,1.9,0.484,0.300,0.804,8.7,0.194
7,Kevin Durant,30,GSW,0.0,25.0,1010,0.025,78,34.6,26.0,6.4,5.9,0.7,1.1,0.521,0.353,0.885,11.5,0.204
8,Kawhi Leonard,27,TOR,0.0,13.0,1010,0.013,60,34.0,26.6,7.3,3.3,1.8,0.4,0.496,0.371,0.854,9.5,0.224
9,Russell Westbrook,30,OKC,0.0,8.0,1010,0.008,73,36.0,22.9,11.1,10.7,1.9,0.5,0.428,0.290,0.656,6.8,0.124


In [15]:
scaler = StandardScaler()

In [16]:
scaler.fit(all_curr_data[["age", "g","mp_per_g", "pts_per_g", "trb_per_g", "ast_per_g", "stl_per_g", "blk_per_g", "fg_pct", "fg3_pct", "ft_pct", "ws", "ws_per_48"]])

StandardScaler()

In [17]:
scaled_data = scaler.transform(all_curr_data[["age", "g","mp_per_g", "pts_per_g", "trb_per_g", "ast_per_g", "stl_per_g", "blk_per_g", "fg_pct", "fg3_pct", "ft_pct", "ws", "ws_per_48"]])

In [18]:
params = scaler.get_params

In [19]:
params

<bound method BaseEstimator.get_params of StandardScaler()>

In [20]:
test = pd.read_csv(os.path.join("Data", str(2020) + ".csv"))

In [21]:
def concat(df1, df2):
    frames = [df1, df2]
    return pd.concat(frames, ignore_index=True)

In [22]:
concat_df = pd.read_csv("Data/1956.csv")
for year in range(1957,2021):
    try:
        df = pd.read_csv(os.path.join("Data", str(year) + ".csv"))
        concat_df = concat(concat_df, df)
    except:
        continue

In [23]:
concat_df

,player,age,team_id,votes_first,points_won,points_max,award_share,g,mp_per_g,pts_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,fg_pct,fg3_pct,ft_pct,ws,ws_per_48
0,Bob Pettit,23,STL,33.0,33.0,80,0.413,72,38.8,25.7,16.2,2.6,NaN,NaN,0.429,NaN,0.736,13.8,0.236
1,Paul Arizin,27,PHW,21.0,21.0,80,0.263,72,37.8,24.2,7.5,2.6,NaN,NaN,0.448,NaN,0.810,12.2,0.214
2,Bob Cousy,27,BOS,11.0,11.0,80,0.138,72,38.4,18.8,6.8,8.9,NaN,NaN,0.360,NaN,0.844,6.8,0.119
3,Mel Hutchins,27,FTW,9.0,9.0,80,0.113,66,33.9,12.0,7.5,2.7,NaN,NaN,0.425,NaN,0.643,4.4,0.095
4,Dolph Schayes,27,SYR,2.0,2.0,80,0.025,72,35.0,20.4,12.4,2.8,NaN,NaN,0.387,NaN,0.858,11.8,0.225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831,Damian Lillard,29,POR,0.0,23.0,1010,0.023,66,37.5,30.0,4.3,8.0,1.1,0.3,0.463,0.401,0.888,11.6,0.225
832,Nikola Jokić,24,DEN,0.0,18.0,1010,0.018,73,32.0,19.9,9.7,7.0,1.2,0.6,0.528,0.314,0.817,9.8,0.202
833,Pascal Siakam,25,TOR,0.0,17.0,1010,0.017,60,35.2,22.9,7.3,3.5,1.0,0.9,0.453,0.359,0.792,5.4,0.123
834,Jimmy Butler,30,MIA,0.0,9.0,1010,0.009,58,33.8,19.9,6.7,6.0,1.8,0.6,0.455,0.244,0.834,9.0,0.221


In [24]:
concat_df.fillna(0, inplace=True)

In [25]:
concat_df

,player,age,team_id,votes_first,points_won,points_max,award_share,g,mp_per_g,pts_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,fg_pct,fg3_pct,ft_pct,ws,ws_per_48
0,Bob Pettit,23,STL,33.0,33.0,80,0.413,72,38.8,25.7,16.2,2.6,0.0,0.0,0.429,0.000,0.736,13.8,0.236
1,Paul Arizin,27,PHW,21.0,21.0,80,0.263,72,37.8,24.2,7.5,2.6,0.0,0.0,0.448,0.000,0.810,12.2,0.214
2,Bob Cousy,27,BOS,11.0,11.0,80,0.138,72,38.4,18.8,6.8,8.9,0.0,0.0,0.360,0.000,0.844,6.8,0.119
3,Mel Hutchins,27,FTW,9.0,9.0,80,0.113,66,33.9,12.0,7.5,2.7,0.0,0.0,0.425,0.000,0.643,4.4,0.095
4,Dolph Schayes,27,SYR,2.0,2.0,80,0.025,72,35.0,20.4,12.4,2.8,0.0,0.0,0.387,0.000,0.858,11.8,0.225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831,Damian Lillard,29,POR,0.0,23.0,1010,0.023,66,37.5,30.0,4.3,8.0,1.1,0.3,0.463,0.401,0.888,11.6,0.225
832,Nikola Jokić,24,DEN,0.0,18.0,1010,0.018,73,32.0,19.9,9.7,7.0,1.2,0.6,0.528,0.314,0.817,9.8,0.202
833,Pascal Siakam,25,TOR,0.0,17.0,1010,0.017,60,35.2,22.9,7.3,3.5,1.0,0.9,0.453,0.359,0.792,5.4,0.123
834,Jimmy Butler,30,MIA,0.0,9.0,1010,0.009,58,33.8,19.9,6.7,6.0,1.8,0.6,0.455,0.244,0.834,9.0,0.221


In [26]:
model = Pipeline([
    ("std", StandardScaler()),
    ("lr", LinearRegression())
])
model.fit(concat_df[["pts_per_g", "trb_per_g", "ast_per_g", "stl_per_g", "blk_per_g", "ws", "ws_per_48"]], concat_df["award_share"])
test["predicted_share"] = model.predict(test[["pts_per_g", "trb_per_g", "ast_per_g", "stl_per_g", "blk_per_g", "ws", "ws_per_48"]])


In [27]:
test.sort_values("predicted_share", ascending=False)

,player,age,team_id,votes_first,points_won,points_max,award_share,g,mp_per_g,pts_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,fg_pct,fg3_pct,ft_pct,ws,ws_per_48,predicted_share
2,James Harden,30,HOU,0.0,367.0,1010,0.363,68,36.5,34.3,6.6,7.5,1.8,0.9,0.444,0.355,0.865,13.1,0.254,0.385493
0,Giannis Antetokounmpo,25,MIL,85.0,962.0,1010,0.952,63,30.4,29.5,13.6,5.6,1.0,1.0,0.553,0.304,0.633,11.1,0.279,0.377990
5,Anthony Davis,26,LAL,0.0,82.0,1010,0.081,62,34.4,26.1,9.3,3.2,1.5,2.3,0.503,0.330,0.846,11.1,0.250,0.274515
7,Damian Lillard,29,POR,0.0,23.0,1010,0.023,66,37.5,30.0,4.3,8.0,1.1,0.3,0.463,0.401,0.888,11.6,0.225,0.273755
1,LeBron James,35,LAL,16.0,753.0,1010,0.746,67,34.6,25.3,7.8,10.2,1.2,0.5,0.493,0.348,0.693,9.8,0.204,0.256741
3,Luka Dončić,20,DAL,0.0,200.0,1010,0.198,61,33.6,28.8,9.4,8.8,1.0,0.2,0.463,0.316,0.758,8.8,0.207,0.248246
4,Kawhi Leonard,28,LAC,0.0,168.0,1010,0.166,57,32.4,27.1,7.1,4.9,1.8,0.6,0.470,0.378,0.886,8.7,0.226,0.185971
8,Nikola Jokić,24,DEN,0.0,18.0,1010,0.018,73,32.0,19.9,9.7,7.0,1.2,0.6,0.528,0.314,0.817,9.8,0.202,0.185000
10,Jimmy Butler,30,MIA,0.0,9.0,1010,0.009,58,33.8,19.9,6.7,6.0,1.8,0.6,0.455,0.244,0.834,9.0,0.221,0.155221
6,Chris Paul,34,OKC,0.0,26.0,1010,0.026,70,31.5,17.6,5.0,6.7,1.6,0.2,0.489,0.365,0.907,8.9,0.193,0.093993


In [28]:
all_curr_data["predicted"] = model.predict(all_curr_data[["pts_per_g", "trb_per_g", "ast_per_g", "stl_per_g", "blk_per_g", "ws", "ws_per_48"]])

In [29]:
all_curr_data.sort_values("predicted", ascending=False)

,Rk,player,age,Team,g,mp_per_g,fg_pct,fg3_pct,ft_pct,trb_per_g,...,stl_per_g,blk_per_g,pts_per_g,PER,USG%,OWS,DWS,ws,ws_per_48,predicted
10,11,Nikola Jokić,25,DEN,64,35.0,0.568,0.412,0.860,10.9,...,1.4,0.7,26.2,31.1,29.2,10.9,3.1,14.0,0.300,0.453679
4,5,Giannis Antetokounmpo,26,MIL,53,33.0,0.568,0.305,0.684,11.2,...,1.2,1.2,28.0,29.1,32.6,5.8,3.1,8.9,0.244,0.278142
2,3,Luka Dončić,21,DAL,57,35.2,0.486,0.356,0.725,8.0,...,1.0,0.6,28.6,26.0,35.8,5.0,2.3,7.4,0.176,0.183562
12,13,Kawhi Leonard,29,LAC,47,34.3,0.517,0.397,0.878,6.6,...,1.6,0.4,25.5,26.8,29.0,6.0,2.1,8.2,0.243,0.183421
0,1,Stephen Curry,32,GSW,56,34.0,0.486,0.426,0.916,5.5,...,1.2,0.1,31.3,26.1,34.0,5.8,2.1,7.9,0.199,0.155845
3,4,Damian Lillard,30,POR,58,35.8,0.435,0.378,0.926,4.2,...,0.9,0.3,28.3,24.5,31.7,7.5,0.6,8.1,0.187,0.147228
6,7,Zion Williamson,20,NOP,59,33.1,0.616,0.313,0.694,7.1,...,0.9,0.6,27.0,27.4,29.8,7.2,1.4,8.6,0.212,0.145060
7,8,Kyrie Irving,28,BRK,47,35.2,0.502,0.384,0.924,4.9,...,1.3,0.6,26.9,24.2,30.5,5.1,1.3,6.4,0.185,0.099679
13,14,Trae Young,22,ATL,56,34.2,0.434,0.359,0.878,3.9,...,0.9,0.2,25.5,22.9,33.0,5.2,1.1,6.3,0.158,0.090344
8,9,Donovan Mitchell,24,UTA,53,33.4,0.438,0.386,0.845,4.4,...,1.0,0.3,26.4,21.3,33.6,4.0,2.2,6.2,0.167,0.041992


In [30]:
model.score(test[["pts_per_g", "trb_per_g", "ast_per_g", "stl_per_g", "blk_per_g", "ws", "ws_per_48"]], test["award_share"])

0.34040509996952517

In [ ]:
all_curr_data["predicted_share"] = lr.predict(all_curr_data[["pts_per_g", "trb_per_g", "ast_per_g", "stl_per_g", "blk_per_g", "ws", "ws_per_48"]])
all_curr_data